In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from pykrx import stock
import mplfinance as mpf

# 예측을 위한 함수
def predict_pattern(model, ohlcv_data, window_size=70):
    """
    외부에서 입력받은 OHLCV 데이터를 기반으로 6개의 차트 패턴에 대한 확률을 예측하는 함수
    :param model: 학습된 CNN-LSTM 모델
    :param ohlcv_data: 새로운 OHLCV 데이터 (numpy 배열 형태로)
    :param window_size: 모델이 사용한 윈도우 크기 (슬라이딩 윈도우 크기)
    :return: 각 패턴에 속할 확률
    """
    # OHLCV 데이터를 정규화 (외부 데이터)
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(ohlcv_data)

    # 입력 데이터를 슬라이딩 윈도우로 변환
    if len(scaled_data) < window_size:
        raise ValueError(f"Input data length should be at least {window_size}")

    input_data = np.array([scaled_data[-window_size:]])  # 최신 데이터로 슬라이딩 윈도우 생성
    
    # 모델 예측 (확률 반환)
    predictions = model.predict(input_data)

    # 6개의 클래스에 대한 확률 출력
    return predictions[0]  # 예측된 확률

# DataFrame 분할 함수
def split_df(df, split_window):
    df_length = len(df)

    if df_length < split_window:
        print(f'{split_window} 기준으로 분할할 수 없습니다! 현재 길이: {df_length}')
    
    final_df_list = []

    for i in range(len(df) - split_window):
        df_start_point = i
        df_end_point = i + split_window
        splited_df = df.iloc[df_start_point:df_end_point]
        final_df_list.append(splited_df)

    return final_df_list    

In [18]:
from tensorflow.keras.models import load_model

# 모델 파일 경로
model_path = '../save_model/chart_pattern_model.h5'

# 모델 불러오기
model = load_model(model_path)

# 모델이 정상적으로 불러와졌는지 확인
print("Model loaded successfully from", model_path)

Model loaded successfully from ../save_model/chart_pattern_model.h5


In [ ]:
start_date = '20240101'
end_date = '20240701'
ticker = '005380'

In [24]:
# 삼성전자 주식 데이터 불러오기 (열 이름 변경)
df = stock.get_market_ohlcv(start_date, end_date, ticker).drop('등락률', axis=1)

# 열 이름을 영어로 변경
df.rename(columns={'시가': 'Open', '고가': 'High', '저가': 'Low', '종가': 'Close', '거래량': 'Volume'}, inplace=True)

df_list = split_df(df, 70)

patterns = ['ascending_triangle', 'descending_triangle', 'ascending_wedge', 'descending_wedge', 'double_top', 'double_bottom']

for i in range(len(df_list)):
    predicted_probabilities = predict_pattern(model, df_list[i])
    which_pattern = list(predicted_probabilities).index(max(predicted_probabilities))

    mpf.plot(df_list[i], type='candle', style='charles', title=f'Prediction: {patterns[which_pattern]} | Prob: {predicted_probabilities[which_pattern]:.4f}', figratio=(12, 8), figscale=1.0)

    # 각 패턴에 속할 확률을 출력
    for pattern, prob in zip(patterns, predicted_probabilities):
        print(f"Pattern: {pattern}, Probability: {prob:.4f}")